In [1]:
# Import 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mlp

In [49]:
# read data
df = pd.read_pickle("../../data/raw/df_resample_100ms.pkl")

In [50]:
df

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label,subject_id,trial,age,height,weight,gender
timestamp,,,,,,,,,,,,,
1970-01-01 00:38:38.700,0.948777,-9.636166,0.002699,0.003818,0.016875,0.006643,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:38.800,0.948993,-9.759188,0.087482,0.020693,0.027565,0.015669,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:38.900,1.071514,-9.787465,-0.093610,0.168721,0.075747,0.037797,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:39.000,1.135679,-9.754036,0.065878,0.339488,0.078970,0.040134,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:39.100,1.126174,-9.443248,0.075006,0.594494,-0.012065,0.103557,BSC,1.0,1.0,32.0,180.0,85.0,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 04:49:55.200,2.128992,14.084110,0.907692,-1.022787,0.162796,0.609964,WAL,67.0,1.0,23.0,180.0,67.0,M
1970-01-01 04:49:55.300,-1.563028,11.139874,1.061856,-2.417346,-0.699792,1.861551,WAL,67.0,1.0,23.0,180.0,67.0,M
1970-01-01 04:49:55.400,2.767679,8.869539,0.154884,-0.998184,0.612194,1.557859,WAL,67.0,1.0,23.0,180.0,67.0,M


## 2. Feature Engineering :

In [51]:
df_temporal = df.copy()

In [52]:
# Compute magnitude features
df_temporal['acc_mag'] = np.sqrt(df['acc_x']**2 + df['acc_y']**2 + df['acc_z']**2)
df_temporal['gyro_mag'] = np.sqrt(df['gyro_x']**2 + df['gyro_y']**2 + df['gyro_z']**2)


### 2.1 Temporal based features

In [53]:
# Specify rolling window (100ms dataset, window_size = 10 means roll over every 1 sec)
window_size = 10
stride = 5  # 50% overlap

In [54]:
from scipy.stats import skew, kurtosis

In [55]:
df_temporal

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label,subject_id,trial,age,height,weight,gender,acc_mag,gyro_mag
timestamp,,,,,,,,,,,,,,,
1970-01-01 00:38:38.700,0.948777,-9.636166,0.002699,0.003818,0.016875,0.006643,BSC,1.0,1.0,32.0,180.0,85.0,M,9.682762,0.018533
1970-01-01 00:38:38.800,0.948993,-9.759188,0.087482,0.020693,0.027565,0.015669,BSC,1.0,1.0,32.0,180.0,85.0,M,9.805610,0.037862
1970-01-01 00:38:38.900,1.071514,-9.787465,-0.093610,0.168721,0.075747,0.037797,BSC,1.0,1.0,32.0,180.0,85.0,M,9.846389,0.188767
1970-01-01 00:38:39.000,1.135679,-9.754036,0.065878,0.339488,0.078970,0.040134,BSC,1.0,1.0,32.0,180.0,85.0,M,9.820149,0.350855
1970-01-01 00:38:39.100,1.126174,-9.443248,0.075006,0.594494,-0.012065,0.103557,BSC,1.0,1.0,32.0,180.0,85.0,M,9.510459,0.603567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 04:49:55.200,2.128992,14.084110,0.907692,-1.022787,0.162796,0.609964,WAL,67.0,1.0,23.0,180.0,67.0,M,14.273005,1.201937
1970-01-01 04:49:55.300,-1.563028,11.139874,1.061856,-2.417346,-0.699792,1.861551,WAL,67.0,1.0,23.0,180.0,67.0,M,11.299000,3.130279
1970-01-01 04:49:55.400,2.767679,8.869539,0.154884,-0.998184,0.612194,1.557859,WAL,67.0,1.0,23.0,180.0,67.0,M,9.292619,1.948866


In [ ]:
 #df_temporal['acc_x_skew'] = df_temporal['acc_x'].rolling(window=window_size, min_periods=1).apply(skew, raw=True)
 
 # This took over 10 min to run just for one columns will try it later with kurtosis to see if it improves modelling

In [56]:
for col in ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'acc_mag', 'gyro_mag']:
    df_temporal[f'{col}_mean'] = df_temporal[col].rolling(window=window_size, min_periods=1).mean()
    df_temporal[f'{col}_std'] = df_temporal[col].rolling(window=window_size, min_periods=1).std()

In [57]:
# Now, subsample every `stride`-th row
df_ws10 = df_temporal.iloc[::stride].reset_index(drop=True).dropna()

In [58]:
df_ws10

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label,subject_id,trial,age,...,gyro_x_mean,gyro_x_std,gyro_y_mean,gyro_y_std,gyro_z_mean,gyro_z_std,acc_mag_mean,acc_mag_std,gyro_mag_mean,gyro_mag_std
1,0.898314,-9.292398,0.379997,0.663339,0.116996,0.078420,BSC,1.0,1.0,32.0,...,0.298425,0.283992,0.050681,0.047834,0.047037,0.037224,9.668136,0.202128,0.312952,0.281789
2,-0.068994,-6.987095,5.288118,1.340131,0.254273,0.553948,BSC,1.0,1.0,32.0,...,0.695667,0.444733,0.131754,0.180522,0.091382,0.191064,9.362781,0.409275,0.742026,0.470523
3,-0.470804,-1.801783,8.929937,1.701367,0.193919,0.328752,BSC,1.0,1.0,32.0,...,1.314200,0.365470,0.097328,0.224595,0.234499,0.221059,9.100286,0.336189,1.375446,0.348528
4,-9.901018,13.441229,7.822155,-0.351828,0.343581,-0.851439,BSC,1.0,1.0,32.0,...,1.725890,0.870151,0.222146,0.295211,-0.066390,0.535790,9.695535,3.126333,1.928122,0.608316
5,-5.150726,6.613287,3.195224,-0.107008,-1.118601,-0.638797,BSC,1.0,1.0,32.0,...,0.694450,1.570145,0.521225,0.734198,-0.122182,0.627285,9.971048,3.118006,1.830100,0.728544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169385,0.586223,5.443400,1.029336,-0.737727,1.443459,1.335214,WAL,67.0,1.0,23.0,...,0.030256,1.296856,-0.401768,0.841217,0.184854,1.546227,10.496543,2.476504,1.955874,0.864953
169386,2.286937,11.130139,-1.195925,1.260658,-0.457721,-0.981431,WAL,67.0,1.0,23.0,...,-0.024428,1.237019,-0.083869,1.068265,0.331298,1.420236,10.523304,3.705045,1.898790,0.901614
169387,-5.076769,14.152212,2.039380,-2.541764,-1.378478,2.412521,WAL,67.0,1.0,23.0,...,0.285874,1.152072,-0.370030,0.987845,-0.172488,1.058050,10.707754,3.534298,1.643457,0.835177
169388,1.671382,11.554947,-1.548809,0.596082,-1.174618,-0.679053,WAL,67.0,1.0,23.0,...,-0.130935,1.246872,-0.387489,0.762758,0.198776,1.346462,10.442834,3.046143,1.749077,0.883200


### Frequency features

1. Power Spectral Density (PSD)
Measures signal power distribution over frequency components.

Useful for identifying dominant energy frequencies in the signal.

In [59]:
df_frequency = df.copy()

In [41]:
from scipy.signal import welch

def compute_psd(series):
    freqs, psd_vals = welch(series, nperseg=len(series), scaling='density')
    return np.sum(psd_vals)  # Total power


In [60]:
df_frequency

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label,subject_id,trial,age,height,weight,gender
timestamp,,,,,,,,,,,,,
1970-01-01 00:38:38.700,0.948777,-9.636166,0.002699,0.003818,0.016875,0.006643,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:38.800,0.948993,-9.759188,0.087482,0.020693,0.027565,0.015669,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:38.900,1.071514,-9.787465,-0.093610,0.168721,0.075747,0.037797,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:39.000,1.135679,-9.754036,0.065878,0.339488,0.078970,0.040134,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:39.100,1.126174,-9.443248,0.075006,0.594494,-0.012065,0.103557,BSC,1.0,1.0,32.0,180.0,85.0,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 04:49:55.200,2.128992,14.084110,0.907692,-1.022787,0.162796,0.609964,WAL,67.0,1.0,23.0,180.0,67.0,M
1970-01-01 04:49:55.300,-1.563028,11.139874,1.061856,-2.417346,-0.699792,1.861551,WAL,67.0,1.0,23.0,180.0,67.0,M
1970-01-01 04:49:55.400,2.767679,8.869539,0.154884,-0.998184,0.612194,1.557859,WAL,67.0,1.0,23.0,180.0,67.0,M


In [ ]:
df_frequency['acc_x_psd'] = df_frequency['acc_x'].apply(compute_psd)